# Laboratorium 4

### Imports

In [25]:
from collections import defaultdict
import re
import math
from typing import Any, Sequence, Dict
import numpy as np
from math import inf
from collections import Counter
from sklearn.cluster import DBSCAN
from time import perf_counter
import pandas as pd



Processing and vectorization

In [3]:
def preprocess(text: str) -> str:
    # Your code here:
    # Convert the text to lowercase.
    # Remove all punctuation marks;
    text = text.lower()
    return re.sub('[^a-z0-9]+', ' ', text)


def text_to_vec(docs: list[str]) -> list[list[int]]:
    # Your code here:
    # Convert documents to numerical vectors.
    # Preprocess documents with the preprocess() function.
    # Represent documents as vectors of word frequencies, 
    # you will need to extract a vocabulary from all the documents.
    freq_vecs = []
    new_docs = []
    for doc in docs:
        new_docs.append(preprocess(doc))
    dictonary = defaultdict(lambda : 0)

    for doc in new_docs:
        words = doc.split()
        for word in words:
            dictonary[word] += 1

    dict_by_id = defaultdict(lambda : 0)

    i = 0

    for key, value in dictonary.items():
        dict_by_id[key] = i
        i += 1

    for doc in new_docs:
        vec = [0 for _ in range(len(dictonary))]
        words = doc.split()
        for word in words:
            vec[dict_by_id[word]] += 1
        freq_vecs.append(vec)
    return freq_vecs

## Zadanie 1 

### Euclidean distance

In [ ]:
def euclidean_distance(text_a: str, text_b: str) -> float:
    # Your code here:
    # Implement the Euclidean distance
    dist = 0
    vec_a, vec_b = text_to_vec([text_a, text_b])
    for i in range(len(vec_a)):
        dist += (vec_a[i] - vec_b[i])**2
    
    return math.sqrt(dist)

### LCS

In [4]:
def lcs(seq_a: Sequence[Any], seq_b: Sequence[Any]) -> int:
    # Your code here:
    # Implement the longest common subsequence calculation.
    # It should work on any sequences, not only on strings.
    len1 = len(seq_a) + 1  # wiersze
    len2 = len(seq_b) + 1  # kolumny
    tab = [[0 for _ in range(len2)] for _ in range(len1)]

    for i in range(len1):
        tab[i][0] = i
    for i in range(len2):
        tab[0][i] = i

    for i in range(1, len1):
        for j in range(1, len2):
            if seq_a[i-1] != seq_b[j-1]:
                tab[i][j] = min(tab[i-1][j-1] + 2, tab[i]
                                [j-1] + 1, tab[i-1][j] + 1)
            else:
                tab[i][j] = min(tab[i-1][j-1], tab[i]
                                [j-1] + 1, tab[i-1][j] + 1)


    return int((len(seq_a) + len(seq_b) - tab[-1][-1])/2)

def word_lcs(text_a: str, text_b: str) -> int:
    # You code here:
    # Using the above function implement the LCS algorithm for texts.
    # Make sure it works on whole words, not on characters.
    seq_a = preprocess(text_a).split()
    seq_b = preprocess(text_b).split()

    return lcs(seq_a, seq_b)

### DICE

In [5]:
def dice(text_a: str, text_b: str) -> float:
    # Your code here:
    # Implement the Dice coefficient
    dice = 0
    docs = [text_a, text_b]
    new_docs = []
    words = []
    for doc in docs:
        new_docs.append(preprocess(doc))

    for doc in new_docs:
        words.append(set(doc.split()))
    
    

    return 2 * len(words[0] & words[1]) / (len(words[0]) + len(words[1]))

### Levenshtein distance

In [6]:
def levenshtein(seq_a: Sequence[Any], seq_b: Sequence[Any]) -> int:
    # Your code here:
    # Implement the Levenshtein distance calculation.
    # It should work on any sequences, not only on strings.

    dist = 0
    len1 = len(seq_a) + 1  # wiersze
    len2 = len(seq_b) + 1  # kolumny
    tab = [[0 for _ in range(len2)] for _ in range(len1)]

    for i in range(len1):
        tab[i][0] = i
    for i in range(len2):
        tab[0][i] = i

    for i in range(1, len1):
        for j in range(1, len2):
            if seq_a[i-1] != seq_b[j-1]:
                tab[i][j] = min(tab[i-1][j-1] + 1, tab[i]
                                [j-1] + 1, tab[i-1][j] + 1)
            else:
                tab[i][j] = min(tab[i-1][j-1], tab[i]
                            [j-1] + 1, tab[i-1][j] + 1)

    # for i in range(len(tab)):
    #     print(tab[i])
    # print("WYNIK")
    return tab[-1][-1]

    


def word_levenshtein(text_a: str, text_b: str) -> int:
    # You code here:
    # Using the above function implement the LCS algorithm for texts.
    # Make sure it works on whole words, not on characters.
    seq_a = preprocess(text_a).split()
    seq_b = preprocess(text_b).split()
    
    return levenshtein(seq_a, seq_b)

### Cosine similarity

In [15]:

    # Your code here:
    # Implement the cosine similarity
    def get_dot_product(vec1, vec2):
        sum = 0
        for i in range(len(vec1)):
            sum += vec1[i] * vec2[i]
        return sum

    def power(x):
        return x*x

    def get_length(vec):
        return math.sqrt(sum(map(power, vec)))


    vec_a, vec_b = text_to_vec([text_a, text_b])
    

    cosine_similarity = get_dot_product(vec_a, vec_b) / (get_length(vec_a) * get_length(vec_b))
    
    return cosine_similarity

## My own testing of functions

In [9]:
def ngrams(text, n=2):
    ngram = {}
    for i in range(len(text) - n + 1):
        if text[i:i + n] in ngram:
            ngram[text[i:i + n]] += 1
        else:
            ngram[text[i:i + n]] = 1
    return ngram

def ngram_length(ngram):
    result = 0
    for key, value in ngram.items():
        result += value ** 2
    return np.sqrt(result)

In [10]:
# cos metric
def cosine_metric(x, y, n=3):
    x_ngrams = ngrams(x, n)
    y_ngrams = ngrams(y, n)
    result = 0
    for key, value in x_ngrams.items():
        if key in y_ngrams:
            result += value * y_ngrams[key]
    if ngram_length(x_ngrams) * ngram_length(y_ngrams) == 0:
        return 1
    return 1 - result / (ngram_length(x_ngrams) * ngram_length(y_ngrams))

In [11]:
text_a = "The quick brown fox jumped over the lazy dog."
text_b = "The lazy dog was jumped over by the quick brown fox."

In [24]:
print(cosine_metric(text_a, text_b))
print((1 -  cosine_similarity(text_a, text_b) )* 2)

0.1524293136404088
0.16026757798440006


___

Preprocess given text by dividing and transforming into d-dimension "vector" 

In [46]:
def vec_from_text(text: str, d: int = 3) -> Dict:
    result = defaultdict(lambda: 0)
    for i in range(len(text) - d + 1):
        result[text[i:i+d]] += 1
    return result

def vec_norm(vect: defaultdict | Any) -> float:
    return math.sqrt(sum(map(lambda x: x*x, vect.values())))

### Cosine similarity

In [52]:
def cosine_similarity(text_a: str, text_b: str, d: int = 3) -> float:
    vec_a = vec_from_text(text_a, d)
    vec_b = vec_from_text(text_b, d)
    result = 0.0

    for key, value in vec_a.items():
        if key in vec_b:
            result += value * vec_b[key]

    if vec_norm(vec_a) * vec_norm(vec_b) == 0:
        return 1
        
    return 1 - result / (vec_norm(vec_a) * vec_norm(vec_b))
    
    # return get_dot_product(vec_a, vec_b) / (vec_norm(vec_a) * vec_norm(vec_b))
    

In [43]:
vect_norm(vect_from_text(text_a))

6.708203932499369

In [53]:
print(cosine_similarity(text_a, text_b))

0.1524293136404088
